<h1>Ensemble Learning</h1>
# setting training set labels to 50/50 %
    <p>160 class 0 samples by adding 70 class 0 records
it should adjust the features of the added function
<br> <ul>adjust the added sample with percentage adjustment</ul>
<h4>score: private 0.638 ; public 0.674</h4>

<h1>step1: <b>Pre-process training data</b></h1>

In [1]:
import random
import numpy as np

def randomInteger(a, b):
    return random.randint(a, b)

def adjustValue(value, percentageValue):    # 0 to xx.x%
    return_value = value
    plusORminus = randomInteger(0, 1) # 0 is plus, 1 is minus

    if (plusORminus == 0):
        return_value += value*(randomInteger(0, percentageValue)/1000)
    else:
        return_value -= value*(randomInteger(0, percentageValue)/1000)

    if (np.isnan(return_value)):
        print("NaN - value*(randomInteger(0, percentageValue)/1000): ", value*(randomInteger(0, percentageValue)/1000))
    
    return return_value 


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression


# df = pd.read_csv('./train.csv', index_col = True)
url = 'https://raw.githubusercontent.com/lucascheng24/comp4432ML-Dont-over-fit-II/main/train.csv'
df = pd.read_csv(url, index_col="id")
df['target'] = df['target'].astype(int)

parameters_dictionary = {}

df_class_1 = df[df['target'] == 1]
df_class_0 = df[df['target'] == 0]

adjusted_percentage_range = 5.5 # best result is 5.5%


print(len(df_class_1), ":", len(df_class_0))

more_class_0 = df_class_0.sample(n=70)
more_class_0 = more_class_0.reset_index()




# random the new class(0) data
index = 0
feature_index = 0

for i in range(len(more_class_0['target'])):
    # print(row)
    for j in range(300):
        more_class_0.at[i, str(j)] = adjustValue(more_class_0.at[i, str(j)], adjusted_percentage_range*10)
        # if (np.isnan(more_class_0.at[i, str(j)])):
        #     print("NaN index;column :", i, ";", j)
        # item[str(i)] = adjustValue(item[str(i)], 100)




    # index = randomInteger(0, 159)
    # feature_index = randomInteger(0, 299)


more_class_0 = more_class_0.drop(columns=['id'])

df_class_0 = pd.concat([df_class_0, more_class_0])

# df_class_0 = more_class_0.reset_index()


df = pd.concat([df_class_0, df_class_1])

# df = df.reset_index()

# print(df)

df.reset_index()

df.rename(columns = {'index': 'id'})

print(len(df_class_1), ":", len(df_class_0))

for element in df:
    if element != "target":
        parameters_dictionary.update({element : df[element]})



label_dictionary = {'label': df['target']}

df_X = pd.DataFrame(data = parameters_dictionary)
df_Y = pd.DataFrame(data = label_dictionary)

# print(df_X['id'])

# applying the method
nan_in_df = df_X.isnull().sum().sum()
 
# printing the number of values present in
# the whole dataframe
print('Number of NaN values present: ' + str(nan_in_df))
# print(df_class_0)






160 : 90
160 : 160
Number of NaN values present: 0


In [4]:
from sklearn import ensemble, preprocessing, metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# initialize the base classifier
dt_clf = DecisionTreeClassifier()

#Create a svm Classifier
svm_clf = SVC(kernel='linear') # Linear Kernel

baggingClassifier = ensemble.BaggingClassifier(estimator = [LogisticRegression(), dt_clf, svm_clf], 
                                               n_estimators = 101, max_features=250, max_samples=200
                                               )
bagging_model = baggingClassifier.fit(df_X, df_Y)





InvalidParameterError: The 'estimator' parameter of BaggingClassifier must be an object implementing 'fit' and 'predict' or None. Got [LogisticRegression(), DecisionTreeClassifier(), SVC(kernel='linear')] instead.

In [ ]:
# implement logistic regression


###
#   function: predict_logistic_regression
#       @param: input_x => training dataset features
#       @param: input_y => training dataset label
#       @param: input_z => testing dataset records (features only)
#   
#   output: array/list of predicted result of input_z
###
model = LogisticRegression(max_iter=10000)

def predict_logistic_regression(input_x, input_y, input_z):
    # build the model based on training data
    model.fit(input_x, input_y)

    # insert testing dataset to get the predicted output
    predicted_values = model.predict(input_z)

    return predicted_values
    


In [ ]:
#  Applying 5-fold-cross-validation
k = 5
kf = KFold(n_splits = k, random_state = None)


# A function to run 5-fold cross validation on polynomial_logistic_regression
def kFold_logistic_regression(input_X, input_Y):

    validation_f1_score = []

    for train_index , validation_index in kf.split(input_X):

        # print('Start model fitting, k=', train_index[0], ' - ', train_index[-1], ' ...')

        X_train , X_valid = input_X.iloc[train_index,:], input_X.iloc[validation_index,:]
        y_train, y_valid = input_Y.iloc[train_index], input_Y.iloc[validation_index]

        # print('X_train: ', X_train)

        y_train = y_train.to_numpy().ravel()
        y_valid = y_valid.to_numpy().ravel()
        
        pred_testValues = predict_logistic_regression(X_train, y_train, X_valid)

        # count the f1 score(true records / predicted records)
        valid_f1 = f1_score(y_valid, pred_testValues)
        validation_f1_score.append(valid_f1)

    return validation_f1_score


In [ ]:
mean_f1 = []

testing_f1_score = kFold_logistic_regression(df_X, df_Y)


# print('testing set F1 score (poly 2 to 12) under 5-fold-cross-validation: ')
for item in testing_f1_score:
    print(item)
    if item != 0.0:
        mean_f1.append(item)
    
np.mean(mean_f1)



<h3>Real prediction on testing data</h3>

In [8]:
import csv


url_test = 'https://raw.githubusercontent.com/lucascheng24/comp4432ML-Dont-over-fit-II/main/test.csv'
df_test = pd.read_csv(url_test, index_col="id")

# print(df_test)

# pred_values = predict_logistic_regression(df_X, df_Y, df_test)
pred_values = bagging_model.predict(df_test)

filePath = '../result_folder/ensembleLearning_v1_1.csv'


print("filePath:", filePath)





filePath: ../result_folder/ensembleLearning_v1_1.csv


In [9]:
with open(filePath, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id", "target"])
    for i in range(250, 20000):
        writer.writerow([i, pred_values[i-250]])
    file.close()